# Data Preprocessing For LaLiga Matches

In this notebook, we will preprocess the football matches dataset to prepare it for analysis and modeling We will perform data cleaning 

In [78]:
import numpy as np
import pandas as pd
import os 

# Load the data and show initial info

In [79]:
# Load the data
df= pd.read_csv(os.path.join("..","raw_data","matches_laliga.csv"))
# Display initial info
print(f"Data shape: {df.shape}")
print(f"rows:{df.shape[0]}, Columns:{df.shape[1]}")
print(df.dtypes)
print("Missing values per column:")
missing_values = df.isnull().sum()
missing_percentage= (missing_values / len(df)) * 100
missing_df = pd.DataFrame({'Missing count': missing_values, 'Percentage': missing_percentage})
print(missing_df[missing_df['Missing count'] > 0].sort_values(by='Missing count', ascending=False))
df.info(show_counts=True)

Data shape: (4700, 29)
rows:4700, Columns:29
Unnamed: 0         int64
date              object
time              object
comp              object
round             object
day               object
venue             object
result            object
gf               float64
ga               float64
opponent          object
xg               float64
xga              float64
poss             float64
attendance       float64
captain           object
formation         object
opp formation     object
referee           object
match report      object
notes            float64
sh               float64
sot              float64
dist             float64
fk               float64
pk                 int64
pkatt              int64
season             int64
team              object
dtype: object
Missing values per column:
            Missing count  Percentage
notes                4700  100.000000
attendance            976   20.765957
dist                    3    0.063830
<class 'pandas.core.frame.DataFrame'>

# Removing irrelevant cols

In [80]:
cols_to_drop=['Unnamed: 0','notes', 'match report']
df=df.drop(columns=[col for col in cols_to_drop if col in df.columns])
print(f"New shape after dropping irrelevant columns: {df.shape}")

New shape after dropping irrelevant columns: (4700, 26)


# Convert date to datetime and extracting features

In [81]:
if 'date' in df.columns:
    df['date']=pd.to_datetime(df['date'], errors='coerce')
    df['year']=df['date'].dt.year
    df['month']=df['date'].dt.month
    df['day_of_week']=df['date'].dt.day_name()
    print("Date column converted to datetime and new features extracted.")
    print(f"Dates covered from {df['date'].min()} to {df['date'].max()}")
    print(f"Years covered: {df['year'].unique()}")

Date column converted to datetime and new features extracted.
Dates covered from 2019-08-16 00:00:00 to 2025-09-30 00:00:00
Years covered: [2025 2024 2023 2022 2021 2020 2019]


# Ensuring numeric cols are proper numeric types

In [82]:
expected_numeric = [
    'gf','ga','xg','xga','poss','attendance',
    'sh','sot','dist','fk','pk','pkatt','season'
]

targets = [c for c in expected_numeric if c in df.columns]
for col in targets:
    df[col] = pd.to_numeric(df[col], errors='coerce')
int_cols = ['gf','ga','pk','pkatt','attendance', 'sh', 'sot', 'fk', 'season']
float_cols=['xg','xga','poss','dist']
for col in int_cols:
    if col in df.columns:
        if df[col].isnull().any():
            df[col] = df[col].astype('float64')
        else:
            df[col]=df[col].astype('Int64')
for col in float_cols:
    if col in df.columns:
        df[col]=df[col].astype('float64')
print("Data types after conversion:")
print(df.dtypes)

Data types after conversion:
date             datetime64[ns]
time                     object
comp                     object
round                    object
day                      object
venue                    object
result                   object
gf                        Int64
ga                        Int64
opponent                 object
xg                      float64
xga                     float64
poss                    float64
attendance              float64
captain                  object
formation                object
opp formation            object
referee                  object
sh                        Int64
sot                       Int64
dist                    float64
fk                        Int64
pk                        Int64
pkatt                     Int64
season                    Int64
team                     object
year                      int32
month                     int32
day_of_week              object
dtype: object


# Fill missing values with median

In [83]:
if 'attendance' in df.columns:
    missing_attendance = df['attendance'].isna().sum()
    print(f"Missing attendance values: {missing_attendance}")
    if missing_attendance>0:
        attendance_median = df['attendance'].median()
        df['attendance'] = df['attendance'].fillna(attendance_median)
        print(f"Filled {missing_attendance} missing attendance values with median: {attendance_median}")
    else:
        print("No missing values in attendance column.")
print("Final data info after preprocessing:")
print(df['attendance'].describe())


Missing attendance values: 976
Filled 976 missing attendance values with median: 20298.5
Final data info after preprocessing:
count     4700.000000
mean     26478.234043
std      16422.898842
min         13.000000
25%      16117.000000
50%      20298.500000
75%      34568.000000
max      95745.000000
Name: attendance, dtype: float64


# Save the clean data 

In [84]:
output_dir=os.path.join("..", "Processed")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
output_path = os.path.join("..","Processed","cleaned_data.csv")
df.to_csv(output_path,index=False)
print(f"Cleaned data saved to {output_path}")

Cleaned data saved to ..\Processed\cleaned_data.csv
